# 7.3. RANS CFD on 2D High-Lift System Configuration Using the Flow360 Python Client

In this tutorial, we will look at a 2D multi-element airfoil which consists of a slat, wing, and flap. We will demonstrate how to create a general multi-element configuration model in ESP and use the Flow360 Python Client to run a RANS simulation. On this page you will find step-by-step instructions on how to:

1. Build a general quasi-3D multi-element configuration model in ESP
2. Set up automatic meshing and execute a RANS simulation
3. Use the Flow360 Python Client to automatically update the configuration and re-submit a case

Firstly, we will explain how to build a general multi-element configuration model in Engineering Sketch Pad (ESP) and how to use edge, face, and group attributes to prepare that model for automatic meshing. Then we will use that general model to build three standard test cases. The first case is a 3-element airfoil that is a cross-section of the NASA CRM-HL configuration and is used in a special session at AIAA Aviation 2020 called [Mesh Effects for CFD Solutions](http://www.gmgworkshop.com/gmgw25.shtml). There is no experimental data for this case and we only compare CFD results. The second case is 30P30N High-Lift Configuration, for which we have experimental data to compare to. The third case is the GA(W)-2 airfoil with a 30% fowler flap for which we also have experimental data.
    
Finally, we will set up the automatic meshing input files and use the Flow360 Python Client to run a RANS simulation. The results of these simulations will then be validated against computational and experimental data.

<div align="center">
<img src="https://docs.flexcompute.com/projects/flow360/en/latest/_images/configuration.png" width="900"/><br>
Fig. 7.3.1 quasi-3D high-lift system configuration generated in ESP
</div>

<br>

<div class="alert alert-block alert-info">
<b>Note</b><br>
In this tutorial, we demonstrate how to build a quasi-3D model for a general high-lift system configuration in ESP. The quasi-3D model here means extrusion of the 2D configuration.
</div>

## 7.3.1. Modeling Multi-element Configurations in ESP
ESP uses the OpenCSM (Open-source Constructive Solid Modeller) system which is a feature-based, associative, parametric solid modeler. More information about ESP installation is available here. To build the geometry through ESP, we use a series of statements and commands in a script. All ESP statements and commands are held in a *.csm file. In this tutorial, we follow a “bottom-up” approach to build a general model for a multi-element configuration including wing, flap and slat. To build the model, we start by defining the design, configuration and constant parameters.

### 7.3.1.1. Defining Geometry Parameters
The design parameter in this tutorial is the flap deflection angle. To define a design parameter, we use **DESPMTR**. Design parameters describe any particular instance of the geometry model. They can be single-valued, 1D vectors, or 2D arrays of numbers. Each design value has a current value, upper- and lower-bounds. ESP allows to compute the sensitivity of any part of a geometry model with respect to any design parameter.

>DESPMTR $pmtrName values\
use:    define a design Parameter

For the multi-element configuration, we also need to define the flap deflection axis location as a configuration parameter. Since this is a quasi-3D model, the span of the quasi-3D model is also a configuration parameter. To define a configuration parameter, we use CFGPMTR. Values for the configuration parameters are declared in the top-level include-type .udc file. They must contain one or more numbers. They must be dimension-ed, in case they are multi-valued. They have lower- and upper-bounds. Their values can be changed and sensitivities cannot be computed for them.

>CFGPMTR    $pmtrName value\
use:    define a configuration Parameter

To build a general model for the multi-element configuration, we need to consider that we may not have flaps, or slats or either. Therefore, their presence in configuration parameters can be defined as a constant parameter for each. To define a constant parameter in a build process, we use CONPMTR. Constant parameters contain only one number. Their values are declared in the top-level include-type .udc file and are visible from any .csm or .udc file. Their values cannot be changed and sensitivities cannot be computed for constant parameters.

>CONPMTR    $pmtrName values\
use:    define a constant Parameter

The example below shows how to put these statements all together in a CSM file.

In this example, DIMENSION indicates the size of the array used to define the flap deflection axis location.

>DIMENSION $pmtrName nrow ncol\
use:    set up or redimensions an array Parameter


>DESPMTR flapDeflection 0\
<mark>DIMENSION flapDefAxis 1 3 1<br>
CFGPMTR flapDefAxis "0.84;0.0;0.0109;"</mark><br>
CFGPMTR span 0.01\
CONPMTR slatStatus 1\
CONPMTR flapStatus 1    

The flap deflection angle is defined in degrees. The span is in meters. The status parameter for the flap and slat indicates if they are present in the configuration. Any value other than zero indicates that they are present in the configuration.

Here you can find the complete CSM file for this step: [multielementStep1.csm](https://simcloud-public-1.s3.amazonaws.com/multielement/multielementStep1.csm).

### 7.3.1.2. 3D Sketches of Multi-element Configuration
In this section, we define a wire body for each element in the configuration. Then, we create a face (e.g. a higher type) from that wire body.

This section is similar to creating 3D sketches for airfoils in the [ONERA M6 Wing Geometry Modeling](https://docs.flexcompute.com/projects/flow360/en/latest/tutorials/ESPTutorial/ESPTutorial.html#om6tutorialsketches). To create a sketch body for each element, we use the following format:

>mark\
<mark>skbeg   up_x_TE  up_y_TE  up_z_TE<br>
spline  up_x_p1  up_y_p1  up_z_p1<br>
.<br>
.<br>
.<br>
spline  up_x_LE  up_y_LE  up_z_LE<br>
skend<br>
<br>
skbeg   low_x_LE  low_y_LE  low_z_LE<br>
spline  low_x_p1  low_y_p1  low_z_p1<br>
.<br>
.<br>
.<br>
spline  low_x_TE  low_y_TE  low_z_TE<br>
skend</mark>\
group\
store airfoil

The above CSM example shows how to put together a 3D sketch for an airfoil. These 3D sketches are later used to build a face body and then face bodies are used to be extruded into a solid body. In this example, we follow a simple concept to define a 2D curve. We define all the curves directly in global 3D space. In case, we need them on a plane, we define 3D data as needed. For example, we define the multi-element configuration on the xz-plane. Therefore, we create all splines in the global 3D space by setting all y-coordinates of the 3D points to zero.

As shown in the above example, all 3D sketches start from the upper trailing-edge point to the leading-edge point and back to the lower trailing-edge point. This means the points definition is counter-clockwise.

In a multi-element configuration, we have three elements including the wing, which is the main element, the slat in front of the wing, and the flap in the back. The wing element profiles can have two curves defined through two splines, when there is a concave region in the back where the flap closes in. It can have more curves when the flap element is present. Since we define a generic multi-element modeler, we include all possible situations. Therefore, wing element profiles always include two curves defined by two splines connected at the leading-edge. If the flap element is present in the configuration, we define a curve using a single spline in a separate sketch for the concave region in the back of the wing.

For the flap element, we have two curves defined by two splines in one sketch. For the slat element, we have two curves: the front and back curves. The front curve is defined with two spline in one sketch. The back curve is defined with one spline in a different sketch. Please note that when we define a single spline in a sketch, we don’t need to group it.

Here you can find the complete CSM file for this step including the sketches for three elements in the configuration: [multielementStep2.csm](https://simcloud-public-1.s3.amazonaws.com/multielement/multielementStep2.csm).

### 7.3.1.3. Building the Wing Face Body

In this section, we **restore** the sketch body for each element and create a face from them.

>RESTORE   $name index=0\
use:    restores Body(s) that was/were previously stored
